In [1]:
import sys
from PyQt5.QtWidgets import QWidget, QApplication, QMainWindow, QDockWidget, QTextEdit, QPlainTextEdit
from PyQt5.QtCore import Qt, QTimer
import filepath
import pdb
from panda3d.core import (
    loadPrcFileData
)
from typing import Tuple
from qpanda3d import QShowBase, QPanda3DWidget, QControl, Synchronizer
from demos.physics_room import RoomScene
from panda3d_game.app import ControlShowBase, ContextShowBase, PhysicsShowBase
from vispyutil.canvas import SynchronizedCanvas
from vispyutil.showbase import CanvasBackgroundShowBase
import numpy as np
from vispy import app, scene, visuals,use
from sympy.physics.units import (
    kilometer, meter,centimeter,
    gram, kilogram, tonne,
    newton, second
)
from util.physics import autocomplete_units, G_val, getG
from demos.ball_room import MassedBall,tmoon
from util.log import *
from config.style import styleSheet
import yaml
import filepath
from util.repo import *
import os
from PyQt5.QtCore import QObject, QEvent
def load_config(config_file):
    with open(config_file, 'r') as file:
        return yaml.safe_load(file)

config = load_config(os.path.join(config_root, "layout_simple.yaml"))
from qtutil.event import *

In [ ]:
# TODO: press : for command
# TODO: console history save and load
# TODO: alias and scripts

import pdb
class StarCanvas(SynchronizedCanvas):
    def __init__(self, n_stars, rho=10):
        SynchronizedCanvas.__init__(self)
        self.n_stars = n_stars 
        # randomly pick stars
        np.random.seed(0)  # todo: use torch random state
        cos_pos_theta = np.random.uniform(-1,1, n_stars)
        pos_theta = np.arccos(cos_pos_theta)
        pos_phi = np.random.uniform(0,2*np.pi, n_stars)
        sizes = np.random.uniform(0, 5, n_stars) 
        r = rho * np.sin(pos_theta)
        x = r * np.cos(pos_phi)
        y = r * np.sin(pos_phi)
        z = rho * np.cos(pos_theta)
        positions = np.vstack([x,y,z]).T
        scatter = scene.visuals.Markers()
        scatter.set_data(positions, edge_color=None, face_color='white', size=sizes, symbol='o')
        # Add scatter plot to the view
        self.view.add(scatter)

class StarScene(CanvasBackgroundShowBase):
    def __init__(self):
        self.stars_canvas = StarCanvas(60000)
        CanvasBackgroundShowBase.__init__(self,self.stars_canvas)

        
class PlanetStarScene(StarScene,PhysicsShowBase):
    def __init__(self):
        StarScene.__init__(self)
        PhysicsShowBase.__init__(self)
        self.bullet_world.setGravity((0,0,0))
        self.unit = {
            "mass" : tonne,
            "length" : 100*meter,
            "time": 1 * second,
            # "force" : sp.Number(1e3) * newton
        }
        autocomplete_units(self.unit)
        self.planet1 = MassedBall(
            name="planet1",
            radius=1000*meter,
            mass=1e6*tonne,
            units=self.unit
        )
        self.planet1.reparentTo(self.render)
        self.planet1.set_texture(tmoon)
        self.planet1.setScale(10)
        self.planet1.setPos(0,10,0)

class PlanetStarSpace(PlanetStarScene, QControl):
    def __init__(self, isQt = True):
        import pdb
        QControl.__init__(self)
        PlanetStarScene.__init__(self)
        self.isQt = isQt
        if self.isQt:
            self.startQt()





from qtutil.qobserver import QObserved


import logging
class QLogHandler(logging.Handler):
    def __init__(self, text_edit: QPlainTextEdit):
        super().__init__()
        self.text_edit = text_edit

    def emit(self, record):
        msg = self.format(record)
        self.text_edit.appendPlainText(msg)

from console import Console

class ConsoleWidget(QPlainTextEdit, Loggable, QObserved):
    def __init__(self, title, console:Console=None):
        QTextEdit.__init__(self,title)
        Loggable.__init__(self,name="console")
        QObserved.__init__(self)
        self.setObjectName("console")
        self.console = console
        self.setPlaceholderText(self.prompt)
        self.prev_cursor = self.cursor_pos
        

    @property
    def prompt(self):
        # TODO： set prompt according to the user
        return "{game prompt} "

    @property
    def cursor_pos(self):
        return self.textCursor().position()
        # TODO: set console

    def mousePressEvent(self, event):
        # return
        pos_bfore_click = self.cursor_pos  # get current cursor
        super().mousePressEvent(event)
        pos_after_click = self.cursor_pos
        if pos_after_click < self.prev_cursor:  
            cursor = self.textCursor()
            cursor.setPosition(pos_bfore_click)  
            self.setTextCursor(cursor)  
        

    def keyPressEvent(self, event):
        if event.key() == Qt.Key_Return or event.key() == Qt.Key_Enter:
            self.put('', refresh_cursor=False, prompt=False)
            if self.console is not None:
                self.debug("handle command, cursor at",self.cursor_pos)
                self.handle_command()
        elif event.key() == Qt.Key_Escape:
            # FIXME: use this only when it is jumped from game to here
            # back to the game
            evt_back_to_game = QEvent(FOCUS_GAME)
            self.send_qevent(evt_back_to_game) # send to all listeners
        else:
            pos_bfore_click = self.cursor_pos
            super().keyPressEvent(event)
            pos_after_click = self.cursor_pos
            if pos_after_click < self.prev_cursor:  
                cursor = self.textCursor()
                cursor.setPosition(pos_bfore_click)  
                self.setTextCursor(cursor)  

    def handle_command(self):
        command = self.toPlainText().strip()[self.prev_cursor:]
        self.debug("command is `{}`".format(command))
        if command:
            Console.parse(self.console, command)
            self.print_output()

    def print_output(self):
        if self.console.out_buffer.empty():
            self.put('') # line break
        while not self.console.out_buffer.empty():
            o = self.console.out_buffer.get()
            self.debug("output is `{}`".format(o))
            self.put(o)

    def put(
        self, s, 
        linebreak=True, 
        refresh_cursor=True,
        prompt=True
    ):
        """
        put new text to widget,
        with predefined behaviour
        """
        if linebreak and not s.endswith('\n'):
            s += '\n'
            if prompt:
                s+=self.prompt
        self.appendPlainText(s)
        if refresh_cursor:
            self.prev_cursor = self.cursor_pos

    def clear(self):
        super().clear(self)
        self.prev_cursor = self.cursor_pos
        

class LoggerWidget(QPlainTextEdit):
    def __init__(self, title):
        QTextEdit.__init__(self,title)
        self.setObjectName("logger")
        self.setReadOnly(True)
        self.handlers = {}

    def add_level(self, level:int):
        if level not in self.handlers.keys():
            new_handler = QLogHandler(self)
            formatter = logging.Formatter(
                # FIXME
                '%(asctime)s - %(levelname)s - %(message)s'
            )
            new_handler.setFormatter(formatter)
            self.handlers[level] = new_handler


class BufferWidget:
    # each item has its type
    # for example, input history or output history
    # each has its str
    pass

class TextBuffer:
    # FIXME: use a queue instead
    # implement: write to a cache file
    # cached queue
    pass

class TextBrowserWidget:
    # browse a buffer
    pass
from typing import Dict
class FocusFilter(QObject):
    def __init__(self, widgets_dict = Dict[int, QWidget]):
        QObject.__init__(self)
        self.widgets_dict = widgets_dict

    def eventFilter(self, obj, event):
        evt_type = event.type()
        if evt_type in self.widgets_dict.keys():
            self.widgets_dict[evt_type].setFocus()
        return super().eventFilter(obj, event)
# MainWindow read layout file

from demos.physics_room import PhyscRoomConsole
class MainWindow(QMainWindow):
    def __init__(self, FPS=60, stylesheet=styleSheet):
        super().__init__()
        self.FPS = FPS
        # Create three dock widgets
        self.dock_top_left = QDockWidget("Game Camera", self)
        self.dock_bottom_left = QDockWidget("Console", self)
        self.dock_right = QDockWidget("Logger", self)
        
        # Add the docks to the main window
        self.addDockWidget(Qt.LeftDockWidgetArea, self.dock_top_left)
        self.addDockWidget(Qt.RightDockWidgetArea, self.dock_right)
        # Split the left dock area vertically (top and bottom)
        self.splitDockWidget(self.dock_top_left, self.dock_bottom_left, Qt.Vertical)
        self.resizeDocks([self.dock_top_left, self.dock_bottom_left], [200, 200], Qt.Vertical)
        self.setWindowTitle("Three Dock Layout")
        self.resize(1600, 1200)
        self.panda3d = None
        self.synchronizer = Synchronizer(self.FPS)
        loadPrcFileData("", "window-type offscreen")
        self.console_widget = ConsoleWidget("")
        self.dock_bottom_left.setWidget(self.console_widget)
        self.log_widget = LoggerWidget("Game Logs")
        self.dock_right.setWidget(self.log_widget)
        self.log_widget.add_level(GAME_LOG)
        Loggable.add_handlers_for_all(self.log_widget.handlers[GAME_LOG]) 
        self.startGame()
        self.panda3d.log("game start")
        self.console = self.get_console()
        self.console_widget.console = self.console
        self.setStyleSheet(styleSheet)
        if stylesheet is not None:
            self.dock_bottom_left.setStyleSheet(stylesheet)
        self.focusFilter = FocusFilter({
            FOCUS_CONSOLE:self.console_widget,
            FOCUS_GAME:self.pandaWidget
        })
        self.installEventFilter(self.focusFilter)
        self.console_widget.register_qobs(self)
        self.pandaWidget.register_qobs(self)

    def startGame(self):
        # TODO: use get game()
        self.panda3d = self.get_game()
        self.panda3d.log("create world")
        self.synchronizer.setShowBase(self.panda3d)
        self.pandaWidget = QPanda3DWidget(
            self.panda3d, 
            synchronizer=self.synchronizer
        )
        self.synchronizer.addWidget(self.pandaWidget)
        self.dock_top_left.setWidget(self.pandaWidget)
        self.synchronizer.start()
        self.panda_mouse_watcher = self.panda3d.mouseWatcherNode # FIXME: let a widget watch this
        self.pandaWidget.setFocus()
        
        # self.panda3d.cam_sensitivity = 0.01

    # todo: remove a widget

    def get_game(self):
        raise NotImplementedError

    def get_console(self):
        raise NotImplementedError

    

class SpaceGame(MainWindow):
    def get_game(self):
        return PlanetStarSpace()

    def get_console(self):
        return PhyscRoomConsole(showbase=self.panda3d)
    

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = SpaceGame()
    window.show()
    sys.exit(app.exec_())

Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
/media/ywatcher/ExtDisk1/LDisk/Packages/game_env/env/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1711322746054/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
ord('esc')

TypeError: ord() expected a character, but string of length 3 found

In [4]:
a = QEvent.registerEventType()
b = QEvent.registerEventType()

In [5]:
type(a)

int

In [10]:
import PyQt5
from PyQt5.QtGui import QTextCursor

In [4]:
t = QPlainTextEdit()

In [5]:
t.appendPlainText("hey")

In [6]:
t.toPlainText()

'hey'

In [7]:
c = t.cursor()

In [10]:
c.pos()

PyQt5.QtCore.QPoint(3203, 733)

In [11]:
tc = t.textCursor()

In [13]:
tc.position()

3

In [15]:
tc.setPosition(0)
tc.position()


0

In [11]:
type(config.keys())

dict_keys

In [12]:
dict_keys = type({}.keys())

In [ ]:
class ConfigMainWindow:
    # TODO: create a dock dict
    # create a command of printing that dict
    
    def parse_config(self, config:dict):
        # FIXME: add multiple dock split
        def is_vertical_split(d:Union[dict,str]):
            return isinstance(d,dict) and set(d.keys()) == {'left','right'}
        def is_horizontal_split(d:Union[dict,str]):
            return isinstance(d,dict) and set(d.keys()) == {'top','bot'}
        # TODO: assertion
        def parse_dock(d:Union[dict,str], parent):
            if isinstance(d, str):
                self.docks[parent] = d
            elif is_vertical_split(d):
                # add two docks
                left_dock = QDockWidget() # FIXME
        

    def __init__(self, FPS=60, stylesheet=styleSheet):
        super().__init__()
        self.FPS = FPS
        self.panda3d = None
        self.synchronizer = Synchronizer(self.FPS)
        loadPrcFileData("", "window-type offscreen")
        self.resize(1600, 1200)
        self.config = config
        self.parse_config(config)
        self.docks = {}
        
        # Create three dock widgets
        # self.dock_top_left = QDockWidget("Game Camera", self)
        # self.dock_bottom_left = QDockWidget("Console", self)
        # self.dock_right = QDockWidget("Logger", self)
        
        # # Add the docks to the main window
        # self.addDockWidget(Qt.LeftDockWidgetArea, self.dock_top_left)
        # self.addDockWidget(Qt.RightDockWidgetArea, self.dock_right)
        # # Split the left dock area vertically (top and bottom)
        # self.splitDockWidget(self.dock_top_left, self.dock_bottom_left, Qt.Vertical)
        # self.resizeDocks([self.dock_top_left, self.dock_bottom_left], [200, 200], Qt.Vertical)
        # self.setWindowTitle("Three Dock Layout")
        
        
        
        self.dock_bottom_left.setWidget(ConsoleWidget("Bottom Left Content"))
        self.log_widget = LoggerWidget("Game Logs")
        self.dock_right.setWidget(self.log_widget)
        self.log_widget.add_level(GAME_LOG)
        Loggable.add_handlers_for_all(self.log_widget.handlers[GAME_LOG]) 
        self.startGame()
        self.panda3d.log("game start")
        self.setStyleSheet(styleSheet)
        if stylesheet is not None:
            self.dock_bottom_left.setStyleSheet(stylesheet)

    def startGame(self):
        self.panda3d = PlanetStarSpace()
        self.panda3d.log("create world")
        self.synchronizer.setShowBase(self.panda3d)
        self.pandaWidget = QPanda3DWidget(
            self.panda3d, 
            synchronizer=self.synchronizer
        )
        self.synchronizer.addWidget(self.pandaWidget)
        self.dock_top_left.setWidget(self.pandaWidget)
        self.synchronizer.start()
        self.panda_mouse_watcher = self.panda3d.mouseWatcherNode # FIXME: let a widget watch this
        self.pandaWidget.setFocus()
    
    def create_widget(self, widget_type:str):
        pass
    pass